<a href="https://colab.research.google.com/github/IrinaProkofieva/KnowledgeGrpahCourse/blob/patch-1/Practice/2023/IPKN/Prokofieva_Gritsai/Dostoevskyi_Queries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rdflib
!pip install urllib3

In [ ]:
from rdflib import Graph

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Читаем граф

In [ ]:
# Create an emprty graph
g = Graph()

g.parse('/content/drive/MyDrive/ontology-with-individuals.owl', format="turtle")
print(g.serialize(format='n3'))

# Делаем запросы

In [ ]:
# Кто стал прототипом для наибольшего числа персонажей в романах Достоевского?
most_influenced_person_query = """
  SELECT ?person (count(?character) AS ?number_of_characters) WHERE {
    ?person a :Реальный_человек .
    ?person :является_прототипом ?character .
    ?character :описан_в ?roman .
    ?roman a :Роман .
    :Достоевский_Федор_Михайлович :является_автором ?roman .
  }
  GROUP BY ?person
  ORDER BY DESC(?number_of_characters)
  LIMIT 5
"""

for row in g.query(most_influenced_person_query):
    print(f"{row.person.fragment.replace('_', ' ')} послужил прототипом для {row.number_of_characters} персонажей")

Бочаров Иван Петрович послужил прототипом для 2 персонажей
Суслова Аполлинария Прокофьевна послужил прототипом для 2 персонажей
Достоевская Мария Федоровна послужил прототипом для 2 персонажей
Лаврентьева Аграфена Тимофеевна послужил прототипом для 2 персонажей
Писарев Дмитрий Иванович послужил прототипом для 2 персонажей


In [ ]:
# У какого персонажа наибольшее число прототипов в романах Достоевского?
most_number_of_prototype = """
  SELECT ?character (count(?person) AS ?number_of_prototypes) WHERE {
    ?character a ?type .
    ?type rdfs:subClassOf* :Персонаж .
    ?person :является_прототипом ?character .
    ?person a :Реальный_человек .
    ?character :описан_в ?roman .
    ?roman a :Роман .
    :Достоевский_Федор_Михайлович :является_автором ?roman .
  }
  GROUP BY ?character
  ORDER BY DESC(?number_of_prototypes)
  LIMIT 5
"""

for row in g.query(most_number_of_prototype):
    print(f"{row.character.fragment.replace('_', ' ')} имеет черты {row.number_of_prototypes} людей")

Версилов Андрей Петрович имеет черты 4 людей
Верховенский Петр Степанович имеет черты 4 людей
Липутин Сергей Васильевич имеет черты 3 людей
Ракитин Михаил Осипович имеет черты 3 людей
Карамазов Дмитрий Федорович имеет черты 3 людей


In [ ]:
# Список героев-самоубийц в романах Достоевского
suicides = """
  SELECT ?character ?roman WHERE {
    ?character a ?type .
    ?type rdfs:subClassOf* :Персонаж .
    ?character :причина_смерти "самоубийство" .
    ?character :описан_в ?roman .
    ?roman a :Роман .
    :Достоевский_Федор_Михайлович :является_автором ?roman .
  }
  ORDER BY ?roman
"""

for row in g.query(suicides):
    print(f"{row.character.fragment.replace('_', ' ')} из романа {row.roman.fragment.replace('_', ' ')}")

Матреша из романа Бесы
Кириллов Алексей Нилыч из романа Бесы
Ставрогин Николай Всеволодович из романа Бесы
Смердяков Павел Федорович из романа Братья Карамазовы
Крафт из романа Подросток
Оля из романа Подросток
Ахмакова Лидия из романа Подросток
Мальчик-самоубийца из романа Подросток
Афросиньюшка из романа Преступление и наказание
Филипп из романа Преступление и наказание
Свидригайлов Аркадий Иванович из романа Преступление и наказание


In [ ]:
# Список персонажей, совершивших преступление, у которых прообразы также совершили приступление, в романах Достоевского
crimes_prototype = """
  SELECT ?character ?roman ?char_crime ?person ?real_crime WHERE {
    ?character a ?type .
    ?type rdfs:subClassOf* :Персонаж .
    ?person :является_прототипом ?character .
    ?person a :Реальный_человек .
    ?character :совершил ?char_crime .
    ?char_crime a ?crime_type .
    ?crime_type rdfs:subClassOf* :Преступление .
    ?person :совершил ?real_crime .
    ?real_crime a ?crime_type .
    ?character :описан_в ?roman .
    ?roman a :Роман .
    :Достоевский_Федор_Михайлович :является_автором ?roman .
  }
  ORDER BY ?roman
"""

for row in g.query(crimes_prototype):
    print(f"{row.character.fragment.replace('_', ' ')} из романа {row.roman.fragment.replace('_', ' ')} совершил {row.char_crime.fragment.replace('_', ' ')}, а его прообраз, {row.person.fragment.replace('_', ' ')}, совершил {row.real_crime.fragment.replace('_', ' ')}")

Виргинский из романа Бесы совершил Убийство Шатова, а его прообраз, Кузнецов Алексей Кириллович, совершил Убийство студента Иванова
Верховенский Петр Степанович из романа Бесы совершил Убийство Шатова, а его прообраз, Нечаев Сергей Геннадьевич, совершил Убийство студента Иванова
Липутин Сергей Васильевич из романа Бесы совершил Убийство Шатова, а его прообраз, Успенский Пётр Гаврилович, совершил Убийство студента Иванова
Виргинский из романа Бесы совершил Убийство Шатова, а его прообраз, Успенский Пётр Гаврилович, совершил Убийство студента Иванова
Рогожин Парфен Семенович из романа Идиот совершил Убийство Настасьи Филипповны, а его прообраз, Мазурин В. Ф., совершил Убийство ювелира 1867


In [ ]:
# Количество экранизаций романов Достоевского
most_screened_roman_query = """
  SELECT ?roman (count(?movie) AS ?number_of_movies) WHERE {
    ?roman a :Роман .
    :Достоевский_Федор_Михайлович :является_автором ?roman .
    ?movie :снят_по_сюжету ?roman .
  }
  GROUP BY ?roman
  ORDER BY DESC(?number_of_movies)
"""

for row in g.query(most_screened_roman_query):
    print(f"Роман {row.roman.fragment.replace('_', ' ')} экранизирован {row.number_of_movies} раз")

Роман Братья Карамазовы экранизирован 19 раз
Роман Преступление и наказание экранизирован 18 раз
Роман Бесы экранизирован 9 раз
Роман Идиот экранизирован 5 раз
Роман Подросток экранизирован 1 раз


In [ ]:
# Топ 7 актеров, снявшийся более, чем в одном фильме по романам Достоевского
actor_with_multiple_movies_query = """
  SELECT ?actor ?number_of_movies ?list_of_movies (group_concat(?character; separator=', ') AS ?list_of_characters)
  WHERE {
    ?actor :играл_персонажа ?character
    {
      SELECT ?actor (count(?movie) AS ?number_of_movies) (group_concat(concat(?name, " (", str(?year), ")"); separator=', ') AS ?list_of_movies)
      WHERE {
        ?roman a :Роман .
        :Достоевский_Федор_Михайлович :является_автором ?roman .
        ?movie :снят_по_сюжету ?roman .
        ?actor :снимался_в ?movie .
        ?movie :название ?name .
        ?movie :год_создания ?year .
      }
      GROUP BY ?actor
      HAVING (count(?movie) > 1)
    }
  }
  GROUP BY ?actor
  ORDER BY DESC(?number_of_movies)
  LIMIT 7
"""

for row in g.query(actor_with_multiple_movies_query):
    print(f"{row.actor.fragment.replace('_', ' ')} снялся в {row.number_of_movies} экранизациях: {row.list_of_movies} и сыграл {', '.join([ch.split('#')[-1].replace('_', ' ') for ch in row.list_of_characters.split(', ')])}")

Евгений Меркурьев снялся в 3 экранизациях: Преступление и наказание (2007), Преступление и наказание (2007), Братья Карамазовы (2008) и сыграл старец Зосима
Франко Скандурра снялся в 2 экранизациях: Братья Карамазовы (1947), Братья Карамазовы (1969) и сыграл Перхотин Петр Ильич, Герценштубе
Евгений Миронов снялся в 2 экранизациях: Идиот (2003), Карамазовы и ад (1996) и сыграл Карамазов Иван Федорович, Мышкин Лев Николаевич
Эме Кларион снялся в 2 экранизациях: Преступление и наказание (1935), Братья Карамазовы (1931) и сыграл Карамазов Иван Федорович, Лужин Петр Петрович
Фоско Джакетти снялся в 2 экранизациях: Братья Карамазовы (1947), Братья Карамазовы (1969) и сыграл Самсонов Кузьма Кузьмич, Карамазов Дмитрий Федорович
Сергей Андрейчук снялся в 2 экранизациях: Преступление и наказание (2007), Преступление и наказание (2007) и сыграл Порох Илья Петрович
Мария Шалаева снялся в 2 экранизациях: Бесы (2014), Братья Карамазовы (2008) и сыграл Лебядкина Марья Тимофеевна


In [ ]:
# Экранизации романов Достоевского с лучшим рейтингом
best_movies_query = """
  SELECT ?name ?year ?rating WHERE {
    ?roman a :Роман .
    :Достоевский_Федор_Михайлович :является_автором ?roman .
    ?movie :снят_по_сюжету ?roman .
    ?movie :название ?name .
    ?movie :рейтинг ?rating .
    ?movie :год_создания ?year .
  }
  ORDER BY DESC(?rating)
  LIMIT 5
"""

for row in g.query(best_movies_query):
    print(f"{row.name} ({row.year}) рейтинг: {row.rating}")

Идиот (2003) рейтинг: 8.5
Преступление и наказание (1969) рейтинг: 8.2
Идиот (1991) рейтинг: 8.1
Идиот (1958) рейтинг: 8.1
Подросток (1983) рейтинг: 8.1


In [ ]:
# Страны, в которых сняты экранизации романов Достоевского
country_movies_query = """
  SELECT ?country (count(?movie) AS ?number_of_movies) WHERE {
    ?roman a :Роман .
    :Достоевский_Федор_Михайлович :является_автором ?roman .
    ?movie :снят_по_сюжету ?roman .
    ?movie :снят_в_стране ?country .
  }
  GROUP BY ?country
  ORDER BY DESC(?number_of_movies)
"""

for row in g.query(country_movies_query):
    print(f"В стране {row.country.fragment.replace('_', ' ')} снято {row.number_of_movies} экранизаций")

В стране Россия снято 15 экранизаций
В стране Франция снято 8 экранизаций
В стране США снято 7 экранизаций
В стране СССР снято 6 экранизаций
В стране Польша снято 4 экранизаций
В стране Германия снято 3 экранизаций
В стране Великобритания снято 3 экранизаций
В стране Италия снято 2 экранизаций
В стране Индия снято 1 экранизаций
В стране Австрия снято 1 экранизаций
В стране Югославия снято 1 экранизаций
В стране Швеция снято 1 экранизаций
В стране Финляндия снято 1 экранизаций
В стране Российская империя снято 1 экранизаций
В стране Иран снято 1 экранизаций
В стране Канада снято 1 экранизаций
В стране Чехия снято 1 экранизаций
В стране Япония снято 1 экранизаций
В стране Бельгия снято 1 экранизаций
